In [ ]:
Aurora Forecasting - Part 04: Actionable Batch Inference

In [ ]:
🗒️ This notebook is divided into the following sections:
Load the trained Kp prediction model from the Hopsworks Model Registry.

Retrieve the most recent solar wind features.

Predict the current Kp index.

Retrieve the latest cloud cover for Stockholm, Luleå, and Kiruna.

Apply city-specific logic to determine if an Aurora is truly visible.

In [ ]:
📝 Imports and Setup

In [ ]:
import hopsworks
import joblib
import pandas as pd
import datetime
from config import HopsworksSettings
import util

# Setup settings
settings = HopsworksSettings()

# Login to Hopsworks
project = hopsworks.login(
    project=settings.HOPSWORKS_PROJECT,
    api_key_value=settings.HOPSWORKS_API_KEY.get_secret_value()
)
fs = project.get_feature_store()
mr = project.get_model_registry()

📂 Step 1: Load the Registered Model

We retrieve the latest version of our Random Forest model from the registry to make our geomagnetic predictions.

In [ ]:
# Get the model object from the registry
model_obj = mr.get_model(
    name=settings.MODEL_NAME,
    version=settings.MODEL_VERSION
)

# Download the model to a local directory
model_dir = model_obj.download()

# Load the model into memory
model = joblib.load(f"{model_dir}/model.pkl")

print(f"Successfully loaded {settings.MODEL_NAME} version {model_obj.version}")

🛰️ Step 2: Retrieve Latest Features and Predict Kp

We fetch the most recent minute-by-minute data from NOAA. In a production environment, you would typically pull the latest data from the Online Feature Store.

In [ ]:
print("Fetching latest solar wind features...")

# Get real-time satellite data
latest_solar_df = util.get_noaa_realtime_data(
    settings.NOAA_MAG_URL,
    settings.NOAA_PLASMA_URL
)

# Use the most recent observation for the prediction
# Features: bx_gsm, by_gsm, bz_gsm, density, speed
latest_features = latest_solar_df[['bx_gsm', 'by_gsm', 'bz_gsm', 'density', 'speed']].tail(1)

# Predict the planetary Kp index
predicted_kp = model.predict(latest_features)[0]

print(f"\n>>> Current Predicted Global Kp Index: {predicted_kp:.2f}")

🏙️ Step 3: Local Visibility and Actionable Decisions

Now we iterate through our three cities. For each, we fetch the cloud cover and apply the "Go/No-Go" logic based on the city's specific Kp threshold.

In [ ]:
results = []
current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

print(f"Analyzing visibility for {current_time}...\n")

for city, city_conf in settings.CITIES.items():
    # 1. Get current cloud cover for this city
    cloud_cover = util.get_city_weather_forecast(
        city_conf['lat'],
        city_conf['lon']
    )

    # 2. Apply the visibility logic (uses thresholds from config.py)
    # Thresholds: Kiruna (1.5), Luleå (3.0), Stockholm (5.0)
    status = util.aurora_visibility_logic(
        pred_kp=predicted_kp,
        kp_threshold=city_conf['kp_threshold'],
        cloud_cover=cloud_cover
    )

    results.append({
        "City": city,
        "Kp Threshold": city_conf['kp_threshold'],
        "Cloud %": cloud_cover,
        "AURORA STATUS": status
    })

# Display the Actionable Results
inference_df = pd.DataFrame(results)
print(inference_df.to_string(index=False))

📊 Step 4: Monitoring and Hindcast (Optional)

Following your Air Quality framework, you can now save these predictions into a "Monitoring" Feature Group to compare your "GO" signals against actual ground-truth reports later.

In [ ]:
# Create or get a monitoring feature group
monitor_fg = fs.get_or_create_feature_group(
    name="aurora_monitoring_fg",
    version=1,
    primary_key=['city', 'time'],
    description="Actual vs Predicted visibility signals for monitoring",
    online_enabled=True
)

# Prepare data for upload
monitoring_data = inference_df.copy()
monitoring_data['time'] = current_time
monitoring_data['predicted_kp'] = predicted_kp

# Insert into Hopsworks
monitor_fg.insert(monitoring_data)

print("\nBatch inference complete and results stored for monitoring.")